In [ ]:
%pip install -qU langchain-community langgraph langchain-openai langchain-anthropic

In [40]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "{your_langsmith_api_key}"
os.environ["OPENAI_API_KEY"] = "{your_openai_api_key}"
os.environ["ANTHROPIC_API_KEY"] = "{your_anthropic_api_key}"

## Quickstart - Single Dummy Tool Case

In [43]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver

# Create the agent
openai_model = ChatOpenAI(model_name="gpt-4o-mini")
anthropic_model = ChatAnthropic(model_name="claude-3-5-haiku-latest")

In [44]:
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]
anthropic_model.invoke(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'id': 'msg_01RueTQXfZ2xjqtJDH7nW89W', 'model': 'claude-3-5-haiku-20241022', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 25, 'output_tokens': 11, 'server_tool_use': None}, 'model_name': 'claude-3-5-haiku-20241022'}, id='run--1c34a26a-e70d-4864-b6de-b19fbc7bef07-0', usage_metadata={'input_tokens': 25, 'output_tokens': 11, 'total_tokens': 36, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [23]:
@tool
def get_weather(city: str) -> int:
    """Get weather of the city."""
    return f"Weather of {city} is sunny"


tools = [get_weather]

In [24]:
memory = MemorySaver()
system_message = "You are a helpful assistant."

agent_executor = create_react_agent(openai_model, tools, prompt=system_message, checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "test-thread"}} # Need explanation on this

In [26]:
agent_output = agent_executor.invoke(
    {
        "messages": [
            ("user", "What's the weather of Paris")
        ]
    },
    config,
)
print(agent_output)

## Build Airline Service Agent

In [28]:
from pydantic import BaseModel

class Date(BaseModel):
    # Somehow LLM is bad at specifying `datetime.datetime`
    year: int
    month: int
    day: int
    hour: int

class UserProfile(BaseModel):
    user_id: str
    name: str
    email: str

class Flight(BaseModel):
    flight_id: str
    date_time: Date
    origin: str
    destination: str
    duration: float
    price: float

class Itinerary(BaseModel):
    confirmation_number: str
    user_profile: UserProfile
    flight: Flight

class Ticket(BaseModel):
    user_request: str
    user_profile: UserProfile

In [29]:
user_database = {
    "Adam": UserProfile(user_id="1", name="Adam", email="adam@gmail.com"),
    "Bob": UserProfile(user_id="2", name="Bob", email="bob@gmail.com"),
    "Chelsie": UserProfile(user_id="3", name="Chelsie", email="chelsie@gmail.com"),
    "David": UserProfile(user_id="4", name="David", email="david@gmail.com"),
}

flight_database = {
    "DA123": Flight(
        flight_id="DA123",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=1),
        duration=3,
        price=200,
    ),
    "DA125": Flight(
        flight_id="DA125",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=7),
        duration=9,
        price=500,
    ),
    "DA456": Flight(
        flight_id="DA456",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=1),
        duration=2,
        price=100,
    ),
    "DA460": Flight(
        flight_id="DA460",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=9),
        duration=2,
        price=120,
    ),
}

itinery_database = {}
ticket_database = {}

In [30]:
import random
import string

@tool
def fetch_flight_info(date: Date, origin: str, destination: str):
    """Fetch flight information from origin to destination on the given date"""
    flights = []

    for flight_id, flight in flight_database.items():
        if (
            flight.date_time.year == date.year
            and flight.date_time.month == date.month
            and flight.date_time.day == date.day
            and flight.origin == origin
            and flight.destination == destination
        ):
            flights.append(flight)
    return flights


@tool
def fetch_itinerary(confirmation_number: str):
    """Fetch a booked itinerary information from database"""
    return itinery_database.get(confirmation_number)


@tool
def pick_flight(flights: list[Flight]):
    """Pick up the best flight that matches users' request."""
    sorted_flights = sorted(
        flights,
        key=lambda x: (
            x.get("duration") if isinstance(x, dict) else x.duration,
            x.get("price") if isinstance(x, dict) else x.price,
        ),
    )
    return sorted_flights[0]


def _generate_id(length=8):
    chars = string.ascii_lowercase + string.digits
    return "".join(random.choices(chars, k=length))


@tool
def book_itinerary(flight: Flight, user_profile: UserProfile):
    """Book a flight on behalf of the user."""
    confirmation_number = _generate_id()
    while confirmation_number in itinery_database:
        confirmation_number = _generate_id()
    itinery_database[confirmation_number] = Itinerary(
        confirmation_number=confirmation_number,
        user_profile=user_profile,
        flight=flight,
    )
    return confirmation_number, itinery_database[confirmation_number]


@tool
def cancel_itinerary(confirmation_number: str, user_profile: UserProfile):
    """Cancel an itinerary on behalf of the user."""
    if confirmation_number in itinery_database:
        del itinery_database[confirmation_number]
        return
    raise ValueError("Cannot find the itinerary, please check your confirmation number.")


@tool
def get_user_info(name: str):
    """Fetch the user profile from database with given name."""
    return user_database.get(name)


@tool
def file_ticket(user_request: str, user_profile: UserProfile):
    """File a customer support ticket if this is something the agent cannot handle."""
    ticket_id = _generate_id(length=6)
    ticket_database[ticket_id] = Ticket(
        user_request=user_request,
        user_profile=user_profile,
    )
    return ticket_id


In [33]:
AIRLINE_AGENT_SYSTEM_MESSAGE = """
You are an AI customer service agent for Awesome airline, an airline company that runs flights across the globe. Your
job is to help users book flights and manage iternerary, including canceling and modifying. When the user request cannot
be resolved, make sure you raise a custom support ticket. For the message you return to the user, please include the
confirmation number if a flight is booked, an a custom support ticket number if a ticket is raised. On other scenarios,
please also make sure that all information users need is included.

Your core principles for interacting with users are:

*   **Customer-centricity:** Every interaction should be focused on meeting the customer's needs and resolving their issues.
*   **Accuracy:** Ensure all information provided is factually correct and up-to-date, referencing provided tools whenever possible.
*   **Efficiency:** Aim to resolve customer issues quickly and effectively, minimizing the need for escalation.
*   **Professionalism:** Maintain a courteous and professional tone throughout the conversation.
*   **Empathy:** Acknowledge the customer's frustration and show understanding when appropriate.
"""

In [45]:
memory = MemorySaver()

tools = [
    fetch_flight_info,
    fetch_itinerary,
    pick_flight,
    book_itinerary,
    cancel_itinerary,
    get_user_info,
    file_ticket,
]

agent_executor = create_react_agent(
    model=anthropic_model,
    tools=tools,
    prompt=AIRLINE_AGENT_SYSTEM_MESSAGE,
    checkpointer=memory,
)

In [46]:
config = {"configurable": {"thread_id": "test-thread"}} # Need explanation on this

In [47]:
# It has automatic cache
result = agent_executor.invoke(
    {
        "messages": [
            ("user", "please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam")
        ]
    },
    config,
)

In [49]:
for step in agent_executor.stream(
    {
        "messages": [
            ("user", "please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam")
        ]
    },
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam
================================== Ai Message ==================================

[{'text': "I apologize, but it seems you've already booked a flight with these exact details in our previous interaction. Let me help you confirm your existing itinerary.", 'type': 'text'}, {'id': 'toolu_01Sgxr8WSfeBWzDbLvKqF4jP', 'input': {'origin': 'SFO', 'destination': 'JFK', 'date': {'year': 2025, 'month': 9, 'day': 1, 'hour': 12}}, 'name': 'fetch_flight_info', 'type': 'tool_use'}]
Tool Calls:
  fetch_flight_info (toolu_01Sgxr8WSfeBWzDbLvKqF4jP)
 Call ID: toolu_01Sgxr8WSfeBWzDbLvKqF4jP
  Args:
    origin: SFO
    destination: JFK
    date: {'year': 2025, 'month': 9, 'day': 1, 'hour': 12}
================================= Tool Message =================================
Name: fetch_flight_info

[Flight(flight_id='DA123', date_time=Date(year=2025, mo

In [51]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="please help me book a flight from SFO to JFK on 09/01/2025, my name is Chelsie")]},
    config,
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I'll| help| you book| a flight from San| Francisco (|SFO) to| New York (JFK|) on September 1|st, 2025|,| for| Chelsie.|

First|, I|'ll fetch| the available flight| information|:|I|'ll| help| you pick| the| best| flight:|Now|, I'll retrieve| your| user| information:|Great|!| I|'ll| procee|d with booking the| flight for you:|Excellent|! I|'ve bo|oked your| flight.| Here| are the details:|
-| Confirmation Number|: 2fg|f1zt2|
- Flight|: DA123
-| Date|:| September| 1st|, 2025,| at| 1:00 |AM|
- Route|: SFO to| JFK
- Duration|: 3 hours|
- Price: $|200.|00

The| flight has| been bo|oked under| your name, Ch|elsie, and a| confirmation has| been sent to your email| (|chelsie@gmail|.com).| Is| there anything else I can| help| you| with today|?|

## Multi-stage Program

In [62]:
from typing_extensions import List, TypedDict
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from langgraph.graph import START, StateGraph, END

class InputState(TypedDict):
    question: str

class OutputState(TypedDict):
    answer_and_eval: tuple[str, int]

class OverallState(TypedDict):
    question: str
    answer: str
    eval: int

class Eval(BaseModel):
    eval: int


def generate_answer(state: InputState) -> OverallState:
    prompt_template = ChatPromptTemplate([
        ("system", "You are a helpful assistant"),
        ("user", "{question}")
    ])
    messages = prompt_template.invoke({"question": state["question"]})
    return {
        "question": state["question"],
        "answer": openai_model.invoke(messages).content
    }

def get_eval(state: OverallState) -> OverallState:
    prompt_template = ChatPromptTemplate([
        ("system", "Evaluate the quality of the answer to the user question, in a scale between 0 to 10. Just give me the integer, no explanation needed."),
        ("user", "The question: {question}, and the generated answer: {answer}")
    ])
    messages = prompt_template.invoke({
        "question": state["question"],
        "answer": state["answer"],
    })
    output = int(openai_model.invoke(messages).content)

    return {
        "question": state["question"],
        "answer": state["answer"],
        "eval": output,
    }

def combine(state: OverallState) -> OutputState:
    return {"answer_and_eval": (state["answer"], state["eval"])}

builder = StateGraph(OverallState,input=InputState,output=OutputState)
builder.add_node("node_1", generate_answer)
builder.add_node("node_2", get_eval)
builder.add_node("node_3", combine)
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

In [63]:
graph = builder.compile()
graph.invoke({"question":"What is so great about basketball?"})

{'answer_and_eval': ('Basketball is a beloved sport around the world for many reasons, making it "great" in various aspects. Here are some of the key points:\n\n1. **Accessibility**: Basketball can be played almost anywhere, in backyards, parks, or indoor gyms, with just a hoop and a ball. This makes it accessible to people of all ages and backgrounds.\n\n2. **Teamwork and Camaraderie**: The sport emphasizes teamwork and collaboration. Players must work together, developing bonds and friendships that can last a lifetime.\n\n3. **Physical Fitness**: Playing basketball is a fantastic way to improve cardiovascular health, strength, agility, and coordination. It encourages a healthy, active lifestyle.\n\n4. **Excitement and Entertainment**: The fast-paced nature of basketball, with its high-scoring, quick transitions, and highlight-reel plays, makes it exciting to watch and play.\n\n5. **Global Appeal**: Basketball has a massive global following, with leagues and players from all around th